In [ ]:
# mount the Google Drive
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Normal predictor ##


In [ ]:
#drive/MyDrive/DSA4212/assignment_2/
%cd /content/drive/MyDrive/DSA4212/assignment_2
%ls

/content/drive/MyDrive/DSA4212/assignment_2
 assignment_2_data/               'DSA4212 assignment 2 code'
 dsa4212_2022_assignment_2-2.pdf   tutorial_matrix_factorization_CLASS.ipynb


In [ ]:
import jax as jx
from jax.config import config
config.update('jax_enable_x64', True)  # often needed for LBFGS that requires high-precision


import jax.numpy as jnp
import numpy as onp
import pylab as plt
import tqdm

import imageio
import os
from skimage.transform import rescale, resize, downscale_local_mean
import pandas as pd
import time
import scipy

try:
    import surprise
except ModuleNotFoundError: # Install flax if missing
    !pip install --quiet surprise
    import surprise

In [ ]:
from surprise import NormalPredictor,KNNBasic
from surprise import Dataset
from surprise import Reader
from surprise.accuracy import mse
from surprise.model_selection.validation import cross_validate

In [ ]:
# Reading in data
# anime_id,name,genre,type,rating,members
anime_name = pd.read_csv("assignment_2_data/assignment_2_anime.csv")

# User_id,anime_id,rating
train = pd.read_csv("assignment_2_data/assignment_2_ratings_train.csv")
# User_id,anime_id,rating
test = pd.read_csv("assignment_2_data/assignment_2_ratings_test.csv")

train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4436068 entries, 0 to 4436067
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 101.5 MB


In [ ]:
# Train data
reader = Reader(rating_scale = (1, 10))

train_data = Dataset.load_from_df(train[['user_id', 'anime_id', 'rating']], reader)

trainset = train_data.build_full_trainset()

# Test data
reader = Reader(rating_scale = (1, 10))

test_data = Dataset.load_from_df(test[['user_id', 'anime_id', 'rating']], reader)

testset = test_data.build_full_trainset().build_testset()

In [ ]:
algo = NormalPredictor()
algo.fit(trainset)

In [ ]:
predictions = algo.test(testset)
mse(predictions)

MSE: 4.6107


4.610744216470551